# Modelo de recomendación de juegos

## Importaciones

In [4]:
import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import pyarrow as pa
import pyarrow.parquet as pq

## Datos a utilizar

Se leen los datos que se prepararon luego del EDA y se convierten en dataframe para ser utilizados por el modelo.

In [5]:
modelo = pd.read_parquet('data/10-modelo-recomend.parquet')
item_similar = pd.read_parquet('data/13-item-similar.parquet')

El primer paso es crear un dataframe que contiene los 'user_id' como índices, los juegos ('item_name') como columnas y como valores los ´rating'.

In [4]:
piv = modelo.pivot_table(index=['user_id'], columns=['item_name'], values='rating')
piv

item_name,! That Bastard Is Trying To Steal Our Gold !,0RBITALIS,"10,000,000",100% Orange Juice,1001 Spikes,12 Labours of Hercules,12 Labours of Hercules II: The Cretan Bull,12 is Better Than 6,140,16bit Trader,...,inMomentum,klocki,liteCam Game: 100 FPS Game Capture,oO,planetarian ~the reverie of a little planet~,resident evil 4 / biohazard 4,sZone-Online,the static speaks my name,theHunter,theHunter: Primal
user_id,,,,,,,,,,,,,,,,,,,,,
--000--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ace--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
--ionex--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2SV-vuLB-Kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-Azsael-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuzuga2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zv_odd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zvanik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A continuación, se normalizan los valores del dataframe `piv` restar la media de las calificaciones de un usuario y luego dividir por la diferencia entre el valor máximo y mínimo de las calificaciones. Esto ajusta las calificaciones de un usuario de manera que estén centradas en cero y escaladas en función de su variabilidad. A los usuarios que solo han dado una calificación o han calificado todos los juegos de la misma manera serán eliminados durante este proceso de normalización. Esto se debe a que estos usuarios no aportan información útil para el modelo de recomendación si todas sus calificaciones son iguales o si solo tienen una calificación.

In [5]:
# Normalización del dataframe 'piv'
piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
# Se borran las columnas que contienen solo cero o no tienen rating, se rellenan los vacíos con 0 y se hace la transpuesta
piv_norm.fillna(0, inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:, (piv_norm != 0).any(axis=0)]
piv_norm

user_id,--000--,-Beave-,-I_AM_EPIC-,-SEVEN-,-Thyme-,-kainey9777,00000000000000000001227,00690069006900,03092002,04061993,...,zomgCoBfAce,zoom-the-flash,zoozles,zourock,zrustz16,zsharoarkbr,zuzuga2003,zvanik,zwanzigdrei,zzoptimuszz
item_name,,,,,,,,,,,,,,,,,,,,,
! That Bastard Is Trying To Steal Our Gold !,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0RBITALIS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"10,000,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Orange Juice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001 Spikes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
resident evil 4 / biohazard 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sZone-Online,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
the static speaks my name,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A los datos de esta matriz normalizada se los convierte a un formato de matriz dispersa (sparse matrix) para reducir la memoria utilizada y mejorar la eficiencia en el manejo de grandes conjuntos de datos, especialmente cuando la mayoría de los valores en la matriz son ceros. La matriz dispersa es un tipo de estructura de datos que almacena solo los valores distintos de cero junto con su ubicación en la matriz, en lugar de almacenar todos los valores de la matriz, incluso los ceros.

In [6]:
piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
piv_sparse

<2814x6855 sparse matrix of type '<class 'numpy.float64'>'
	with 24199 stored elements in Compressed Sparse Row format>

Ahora, se crean dos matrices de similitud utilizando la similitud del coseno para medir la similitud entre los juegos (item_similarity) y entre los usuarios (user_similarity).

La similitud del coseno es una medida comúnmente utilizada para evaluar la similitud entre dos vectores en un espacio multidimensional. En el contexto de sistemas de recomendación y análisis de datos, la similitud del coseno se utiliza para determinar cuán similares son dos conjuntos de datos o elementos, y se calcula utilizando el coseno del ángulo entre los vectores que representan esos datos o elementos.

In [7]:
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

Para estructurar y organizar los resultados de manera más accesible y comprensible se insertan las matrices anteriores en un Dataframe.

In [8]:
#item similarity dataframe
item_similar = pd.DataFrame(item_similarity, index = piv_norm.index, columns = piv_norm.index)
#user similarity dataframe
user_similar = pd.DataFrame(user_similarity, index = piv_norm.columns, columns = piv_norm.columns)

## Función de recomendación según un juego

Ahora, conociendo la relación entre los distintos juegos, se puede proponer una función que realice una recomendación de 5 juegos en función de un juego dado, teniendo en cuenta los valores mas altos de similitud del coseno. Esta función toma un nombre de un juego como entrada, luego ordena la columna correspondiente a ese juego en la matriz de similitud entre elementos (`item_similar`) de manera descendente, de modo que los juegos más similares aparezcan en la parte superior. Posteriormente selecciona los 5 juegos más similares (excluyendo el propio juego que se pasó como entrada), itera a través de estos juegos similares y, finalmente, imprime una lista de juegos similares al juego especificado.

In [9]:
def top_game(game):
    '''
    Muestra una lista de juegos similares a un juego dado.

    Args:
        game (str): El nombre del juego para el cual se desean encontrar juegos similares.

    Returns:
        None: Esta función imprime una lista de juegos 5 similares al dado.

    '''
    count = 1
    print('Similar games to {} include:\n'.format(game))
    for item in item_similar.sort_values(by = game, ascending = False).index[1:6]:
        print('No. {}: {}'.format(count, item))
        count +=1  

In [10]:
# ejemplo
top_game('Counter-Strike')

Similar games to Counter-Strike include:

No. 1: Tap Heroes
No. 2: Counter-Strike: Condition Zero
No. 3: S.T.A.L.K.E.R.: Call of Pripyat
No. 4: World of Soccer online
No. 5: Hearts of Iron III


In [11]:
# ejemplo
top_game('Age of Empires II: HD Edition')

Similar games to Age of Empires II: HD Edition include:

No. 1: Farnham Fables
No. 2: Trials Evolution Gold Edition
No. 3: Brütal Legend
No. 4: Serious Sam HD: The First Encounter
No. 5: Darkest Dungeon


## Función de recomendación según un usuario

La siguiente función tiene la finalidad de mostrar una lista de usuarios más similares a un usuario dado, junto con sus valores de similitud. Primero, ordena la matriz de similitud entre usuarios (`user_similar`) en orden descendente según la similitud con el usuario dado ('user_id'), luego toma los 5 usuarios más similares (excluyendo el propio usuario) y almacena sus nombres y valores de similitud en las listas sim_users y sim_values. Finalmente, combina los nombres de usuario y los valores de similitud en una lista de tuplas utilizando la función zip y los imprime.

Esto es útil en sistemas de recomendación para mostrar a un usuario los usuarios más similares en función de sus calificaciones pasadas, lo que puede ayudar en la generación de recomendaciones personalizadas.

In [12]:
def top_users(user):
    '''
    Muestra una lista de los usuarios más similares a un usuario dado y sus valores de similitud.

    Args:
        user (str): El nombre o identificador del usuario para el cual se desean encontrar usuarios similares.

    Returns:
        None: Esta función imprime la lista de usuarios similares y sus valores de similitud en la consola.

    '''
    # Verifica si el usuario está presente en las columnas de piv_norm (si no está, devuelve un mensaje)
    if user not in piv_norm.columns:
        return('No data available on user {}'.format(user))
    
    print('Most Similar Users:\n')
    # Ordena los usuarios por similitud descendente y toma los 5 usuarios más similares (excluyendo el propio 'user')
    sim_values = user_similar.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:6]
    sim_users = user_similar.sort_values(by=user, ascending=False).index[1:11]
    # Combina los nombres de usuario y los valores de similitud en una lista de tuplas
    zipped = zip(sim_users, sim_values,)
    
    # Itera a través de las tuplas y muestra los usuarios similares y sus valores de similitu
    for user, sim in zipped:
        print('User #{0}, Similarity value: {1:.2f}'.format(user, sim)) 

In [22]:
# ejemplo
top_users('76561198028042361')

Most Similar Users:

User #haloslayer1, Similarity value: 0.85
User #76561198112249349, Similarity value: 0.85
User #123123718417671290, Similarity value: 0.79
User #DeNo64, Similarity value: 0.75
User #tsunamitad, Similarity value: 0.75


En esta función, se generar una lista de 5 juegos recomendados para un usuario en función de las calificaciones de usuarios similares. Los juegos que son más frecuentemente recomendados por usuarios similares se consideran como las principales recomendaciones para ese usuario.

In [6]:
def similar_user_recs(user):
    '''
    Genera una lista de los juegos más recomendados para un usuario, basándose en las calificaciones de usuarios similares.

    Args:
        user (str): El nombre o identificador del usuario para el cual se desean generar recomendaciones.

    Returns:
        list: Una lista de los juegos más recomendados para el usuario basado en la calificación de usuarios similares.

    '''
    # Verifica si el usuario está presente en las columnas de piv_norm (si no está, devuelve un mensaje)
    if user not in piv_norm.columns:
        return('No data available on user {}'.format(user))
    
    # Obtiene los usuarios más similares al usuario dado
    sim_users = user_similar.sort_values(by=user, ascending=False).index[1:11]
    
    best = []  # Lista para almacenar los juegos mejor calificados por usuarios similares
    most_common = {}  # Diccionario para contar cuántas veces se recomienda cada juego
    
    # Para cada usuario similar, encuentra el juego mejor calificado y lo agrega a la lista 'best'
    for i in sim_users:
        max_score = piv_norm.loc[:, i].max()
        best.append(piv_norm[piv_norm.loc[:, i]==max_score].index.tolist())
    
    # Cuenta cuántas veces se recomienda cada juego
    for i in range(len(best)):
        for j in best[i]:
            if j in most_common:
                most_common[j] += 1
            else:
                most_common[j] = 1
    
    # Ordena los juegos por la frecuencia de recomendación en orden descendente
    sorted_list = sorted(most_common.items(), key=operator.itemgetter(1), reverse=True)
    
    # Devuelve los 5 juegos más recomendados
    return sorted_list[:5]  

In [21]:
# ejemplo
similar_user_recs('sharks003')

[('Call of Duty: World at War', 10),
 ('Yet Another Zombie Defense', 1),
 ('Call of Duty: Black Ops', 1),
 ('Team Fortress Classic', 1),
 ('They Bleed Pixels', 1)]

In [20]:
# ejemplo
similar_user_recs('ilovehamstersandbunnies')

[('Call of Duty: World at War', 10),
 ('Yet Another Zombie Defense', 1),
 ('Call of Duty: Black Ops', 1),
 ('Company of Heroes 2', 1)]

## Carga de las matrices para recomendar en la API

Para poder utilizar las funciones **top_users** y **similar_user_recs** es necesario consumir las matrices **piv_norm** y **user_sim_df**. Por ello, se guardan, en este caso, en formato parquet los que permite una compresión y codificación eficiente.

In [23]:
pq.write_table(pa.Table.from_pandas(piv_norm), 'data/11-piv-norm.parquet')
pq.write_table(pa.Table.from_pandas(user_similar), 'data/12-user-similar.parquet')
pq.write_table(pa.Table.from_pandas(item_similar), 'data/13-item-similar.parquet')
print('Se guardaron correctamente')

Se guardaron correctamente


In [14]:
def recomendacionJuego(juego):
  '''
  Esta función muestra una lista de juegos similares a un juego dado.

  Parameters:
  ----------
  juego (str): El nombre del juego para el cual se desean encontrar juegos similares.

  Returns:
  ----------
  juegos_similares: Esta función imprime una lista de juegos 5 similares al dado.

  Pasos:
  ----------
  
Verificamos si el juego está en el DataFrame de similitud
Obtenemos la lista de juegos similares y mostrarlos
Imprimimos la lista de juegos similares

  '''

  # Paso 1
  if juego not in item_similar.index:
      print(f'No se encontraron juegos similares para {juego}.')
      return

  # Paso 2
  similar_juegos = item_similar.sort_values(by=juego, ascending=False).index[1:6]  # Mostrar siempre los primeros 5

  # Paso 3
  juegos_similares = [item for item in similar_juegos]

  return juegos_similares


In [13]:
recomendacion_juego('Age of Empires II: HD Edition')

['Tap Heroes',
 'Counter-Strike: Condition Zero',
 'S.T.A.L.K.E.R.: Call of Pripyat',
 'World of Soccer online',
 'Hearts of Iron III']